## 1. 데이터 및 모델 준비

In [14]:
# 데이터 로딩
df = pd.read_csv('../data/processed/selected_features.csv', encoding='utf-8-sig')
target_col = '모형개발용Performance(향후1년내부도여부)'

# 데이터 분리
X = df.drop(columns=[target_col])
y = df[target_col]

# 04번 노트북과 동일한 Split (random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 모델 로딩 (Pipeline 포함됨)
model_path = '../data/processed/final_bankruptcy_model.pkl'
try:
    model = joblib.load(model_path)
    print(f"✅ 모델 로딩 성공: {model_path}")
except FileNotFoundError:
    print(f"❌ 모델 파일을 찾을 수 없습니다. 04번 노트북을 먼저 실행하세요.")

# 예측 (파이프라인이 전처리부터 예측까지 수행)
# X_test(Raw Data)를 그대로 넣습니다.
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

print(f"✅ 예측 완료 (Test Set Size: {len(X_test)})")

✅ 모델 로딩 성공: ../data/processed/final_bankruptcy_model.pkl
✅ 예측 완료 (Test Set Size: 10021)


## 2. Precision-Recall 곡선 및 임계값 최적화

In [15]:
# Precision-Recall 곡선
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

# F2-Score 최적 임계값 찾기
f2_scores = []
# thresholds 길이는 precision/recall보다 1개 짧으므로 맞춤
for i, threshold in enumerate(thresholds):
    prec = precision[i]
    rec = recall[i]
    if (4 * prec + rec) > 0:
        f2 = (5 * prec * rec) / (4 * prec + rec)
    else:
        f2 = 0
    f2_scores.append(f2)

optimal_idx = np.argmax(f2_scores)
optimal_threshold = thresholds[optimal_idx]
optimal_f2 = f2_scores[optimal_idx]

print(f"\n🎯 최적 임계값 (Max F2): {optimal_threshold:.4f}")
print(f"   최적 F2-Score: {optimal_f2:.4f}")
print(f"   해당 Precision: {precision[optimal_idx]:.4f}")
print(f"   해당 Recall: {recall[optimal_idx]:.4f}")

# 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=recall, y=precision,
    mode='lines',
    name=f'PR Curve (AUC={pr_auc:.3f})',
    line=dict(color='blue', width=2)
))
fig.add_trace(go.Scatter(
    x=[recall[optimal_idx]], y=[precision[optimal_idx]],
    mode='markers',
    name=f'Optimal (Th={optimal_threshold:.3f})',
    marker=dict(color='red', size=12, symbol='star')
))
fig.update_layout(
    title='Precision-Recall Curve',
    xaxis_title='Recall',
    yaxis_title='Precision',
    hovermode='x unified'
)
fig.show()


🎯 최적 임계값 (Max F2): 0.0620
   최적 F2-Score: 0.3493
   해당 Precision: 0.1777
   해당 Recall: 0.4605


## 3. 혼동 행렬 및 분류 보고서

In [16]:
# 최적 임계값으로 재예측
y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)

# 혼동 행렬
cm = confusion_matrix(y_test, y_pred_optimal)
tn, fp, fn, tp = cm.ravel()

# 시각화
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['예측: 정상', '예측: 부도'],
    y=['실제: 정상', '실제: 부도'],
    text=[[f'TN<br>{tn}<br>({tn/len(y_test)*100:.1f}%)', 
           f'FP<br>{fp}<br>({fp/len(y_test)*100:.1f}%)'],
          [f'FN<br>{fn}<br>({fn/len(y_test)*100:.1f}%)', 
           f'TP<br>{tp}<br>({tp/len(y_test)*100:.1f}%)']],
    texttemplate='%{text}',
    textfont={"size": 14},
    colorscale='Blues'
))

fig.update_layout(
    title=f'혼동 행렬 (임계값={optimal_threshold:.3f})',
    xaxis_title='예측값',
    yaxis_title='실제값',
    height=500
)
fig.show()

# 상세 메트릭
print("\n📊 상세 성능 지표:")
print("="*60)
print(f"정확도 (Accuracy): {(tp+tn)/len(y_test):.4f}")
print(f"정밀도 (Precision): {tp/(tp+fp):.4f}")
print(f"재현율 (Recall): {tp/(tp+fn):.4f}")
print(f"F1-Score: {2*tp/(2*tp+fp+fn):.4f}")
print(f"F2-Score: {optimal_f2:.4f}")
print(f"\nType I Error (오탐율): {fp/(fp+tn)*100:.2f}%")
print(f"Type II Error (미탐율): {fn/(fn+tp)*100:.2f}%")
print(f"\n부도 기업 중 탐지: {tp}/{tp+fn} ({tp/(tp+fn)*100:.1f}%)")
print(f"정상 기업 중 오탐: {fp}/{fp+tn} ({fp/(fp+tn)*100:.1f}%)")


📊 상세 성능 지표:
정확도 (Accuracy): 0.9595
정밀도 (Precision): 0.1777
재현율 (Recall): 0.4605
F1-Score: 0.2564
F2-Score: 0.3493

Type I Error (오탐율): 3.28%
Type II Error (미탐율): 53.95%

부도 기업 중 탐지: 70/152 (46.1%)
정상 기업 중 오탐: 324/9869 (3.3%)


## 5. 비즈니스 가치 시뮬레이션

In [17]:
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix

def calculate_business_value(y_true, y_pred, cost_matrix):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    total_value = (
        tp * cost_matrix['tp_value'] +
        fp * cost_matrix['fp_cost'] +
        fn * cost_matrix['fn_cost'] +
        tn * cost_matrix['tn_value']
    )
    
    return total_value, {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn}

# 1. 비용 행렬 수정 (TN 이익 추가, 비율 현실화)
# 예: 대출 원금 100, 이자 5, 기회비용 5 가정
cost_matrix = {
    'tp_value': 80,     # 부도 예측 성공 (손실 회피 - 충당금 제외 등)
    'fn_cost': -100,    # 부도 미탐지 (원금 손실)
    'fp_cost': -5,      # 정상 기업 거절 (이자 수익 기회비용)
    'tn_value': 5       # 정상 기업 대출 (이자 수익 5%) - 핵심 추가!
}

# 2. 임계값 탐색 범위 수정 (하한선을 0.01로 낮춤)
thresholds_range = np.linspace(0.01, 0.5, 100)
business_values = []

for threshold in thresholds_range:
    y_pred_temp = (y_pred_proba >= threshold).astype(int)
    value, _ = calculate_business_value(y_test, y_pred_temp, cost_matrix)
    business_values.append(value)

# 최적 임계값 찾기
optimal_business_idx = np.argmax(business_values)
optimal_business_threshold = thresholds_range[optimal_business_idx]
max_business_value = business_values[optimal_business_idx]

print(f"\n💰 비즈니스 가치 최적화 결과:")
print(f"   최적 임계값: {optimal_business_threshold:.3f}")
print(f"   최대 비즈니스 가치: {max_business_value:,.0f}만원")

# 시각화
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=thresholds_range, y=business_values,
    mode='lines',
    name='비즈니스 가치',
    line=dict(color='green', width=2)
))

# 비즈니스 최적 임계값 표시
fig.add_vline(
    x=optimal_business_threshold, 
    line_dash="dash", 
    line_color="red",
    annotation_text=f"<b>Biz Opt: {optimal_business_threshold:.3f}</b>",
    annotation_position="top left"
)

# F2 최적 임계값 표시 (이전 셀에서 구한 optimal_threshold 사용)
if 'optimal_threshold' in locals():
    fig.add_vline(
        x=optimal_threshold, 
        line_dash="dash", 
        line_color="blue",
        annotation_text=f"<b>F2 Opt: {optimal_threshold:.3f}</b>",
        annotation_position="bottom right"
    )

fig.update_layout(
    title='<b>임계값별 비즈니스 가치 시뮬레이션</b>',
    xaxis_title='임계값 (Threshold)',
    yaxis_title='비즈니스 가치 (만원)',
    height=500,
    template='plotly_white'
)

fig.show()

# 비교 분석
y_pred_business = (y_pred_proba >= optimal_business_threshold).astype(int)
business_value_optimal, cm_optimal = calculate_business_value(y_test, y_pred_business, cost_matrix)

# 모델 없이 모두 정상으로 판단(대출 실행)하는 경우
y_pred_naive = np.zeros_like(y_test)
business_value_naive, cm_naive = calculate_business_value(y_test, y_pred_naive, cost_matrix)

print(f"\n📊 비즈니스 가치 비교:")
print(f"   [A] 모델 최적 적용 시: {business_value_optimal:,.0f}만원")
print(f"   [B] 모델 미사용(전원 대출): {business_value_naive:,.0f}만원")
diff = business_value_optimal - business_value_naive
print(f"   🚀 개선 효과 (A - B): {diff:,.0f}만원")


💰 비즈니스 가치 최적화 결과:
   최적 임계값: 0.059
   최대 비즈니스 가치: 43,355만원



📊 비즈니스 가치 비교:
   [A] 모델 최적 적용 시: 43,355만원
   [B] 모델 미사용(전원 대출): 34,145만원
   🚀 개선 효과 (A - B): 9,210만원


## 6. 케이스 스터디 (부도 기업 분석)

## 7. 최종 결과 요약